## CHANGES

- trying L2 loss

## SETUP

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import fastai
from fastai.vision import *
from fastai.callbacks import *

In [ ]:
# show versions
print('fastai:', fastai.__version__)
print('pytorch:', torch.__version__)
print('python:', sys.version.split(' ')[0])

In [ ]:
version = 'v10'       # this should match the notebook filename

seed = 42
arch = models.resnet50
size = 392
bs = 16
num_workers = 6     # set to available cores

scale = 1           # number to divide y by to help normalize values
transform = 'spectogram'  # which time series to visual transformation to use

In [ ]:
# set seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## DATA

Assumes the following has been generated using `gen_spectogram`:
- `X_train.csv`
- `y_train.csv`
- `train_images`
- `test_images`

In [ ]:
# set paths
path = pathlib.Path.home()/'.fastai/data/kaggle_earthquake'
img_path = path/f'train_images/{transform}'
save_path = path/'saved_models'
os.makedirs(save_path, exist_ok=True)

In [ ]:
# load segment CSVs
#pd.options.display.precision = 15
X_train = pd.read_csv(path/'X_train.csv', index_col=0)
y_train = pd.read_csv(path/'y_train.csv', index_col=0)

In [ ]:
#X_train.head()

In [ ]:
y_train.head()

In [ ]:
# graph y_train
print('min:', min(y_train['time_to_failure']))
print('max:', max(y_train['time_to_failure']))
plt.figure()
plt.plot(y_train)

## MODEL

In [ ]:
# scale down the labels
def gen_label(path):
    id = int(path.name.split('_')[1].split('.')[0])
    ttf = y_train.iloc[id]['time_to_failure']
    return ttf / scale

In [ ]:
gen_label(img_path/'seg_0.png')

In [ ]:
# no data augmentation
tfms = get_transforms(do_flip=False, p_affine=0., p_lighting=0.)

In [ ]:
fake_label = 0.
#valid_idx = range(3000, len(y_train))

src = (ImageList.from_folder(img_path)
        .split_by_rand_pct(valid_pct=0.20)
        .label_from_func(gen_label, label_cls=FloatList)
        .add_test_folder(f'../../test_images/{transform}', label=fake_label))

In [ ]:
data = (src.transform(tfms, resize_method=ResizeMethod.SQUISH, size=size)
        .databunch(bs=bs, num_workers=num_workers)
        .normalize(imagenet_stats))

In [ ]:
# verify datasets loaded properly
n_train_items = len(data.train_ds)
n_valid_items = len(data.valid_ds)
n_test_items = len(data.test_ds)

print('train: ', n_train_items)
print('valid: ', n_valid_items)
print('test:  ', n_test_items)
print('')
print('TOTAL: ', n_train_items + n_valid_items + n_test_items)

In [ ]:
# verify images and labels match up
data.show_batch(4, figsize=(9, 9))

In [ ]:
# define metrics
def mae_scaled(preds, targs):
    return mean_absolute_error(preds.view(-1)*scale, targs.view(-1)*scale)

def mse_scaled(preds, targs):
    return mean_squared_error(preds.view(-1)*scale, targs.view(-1)*scale)

In [ ]:
# L1 loss is sum of the all the absolute differences
# more robust to outliers
l1loss = nn.L1Loss()
def l1_loss(preds, targs):
    return l1loss(preds.view(-1), targs.view(-1))

# L2 loss is sum of the all the squared differences
# less robust to outliers
l2loss = nn.MSELoss()
def l2_loss(preds, targs):
    return l2loss(preds.view(-1), targs.view(-1))

In [ ]:
# return correct size of fully connected layer based on pre-trained model
def final_conv_layer_size(arch):
    if arch == models.resnet18 and size == (217, 223): return 512*14*21
    elif arch == models.resnet34 and size == (217, 223): return 512*7*7  
    elif arch == models.resnet50 and size == (217, 223): return 2048*7*7
    elif arch == models.resnet50 and size == 224: return 2048*7*7
    elif arch == models.resnet50 and size == 392: return 2048*13*13

In [ ]:
# define custom heads
small_head = nn.Sequential(Flatten(), nn.Linear(final_conv_layer_size(arch), 1))

small_head_sigmoid = nn.Sequential(Flatten(), nn.Linear(final_conv_layer_size(arch), 1),  nn.Sigmoid())

medium_head = nn.Sequential(
  nn.AvgPool2d(13, 13),
  Flatten(), 
  nn.BatchNorm1d(2048),
  nn.Dropout(0.5),
  nn.Linear(2048, 256),
  nn.ReLU(),
  nn.BatchNorm1d(256),
  nn.Dropout(0.5),
  nn.Linear(256, 1))

big_head = nn.Sequential(
  nn.AvgPool2d(13, 13),
  Flatten(), 
  nn.BatchNorm1d(2048),
  nn.Dropout(0.5),
  nn.Linear(2048, 512),
  nn.ReLU(),
  nn.BatchNorm1d(512),
  nn.Dropout(0.5),
  nn.Linear(512, 128),
  nn.ReLU(),
  nn.BatchNorm1d(128),
  nn.Dropout(0.5),
  nn.Linear(128, 1))


big_head_sigmoid = nn.Sequential(
  nn.AvgPool2d(13, 13),
  Flatten(), 
  nn.BatchNorm1d(2048),
  nn.Dropout(0.5),
  nn.Linear(2048, 512),
  nn.ReLU(),
  nn.BatchNorm1d(512),
  nn.Dropout(0.5),
  nn.Linear(512, 128),
  nn.ReLU(),
  nn.BatchNorm1d(128),
  nn.Dropout(0.5),
  nn.Linear(128, 1),
  nn.Sigmoid())

In [ ]:
# create learner
learn = cnn_learner(data, arch,
                    custom_head=big_head,
                    loss_func=l1_loss,
                    metrics=[mean_squared_error, mean_absolute_error, mse_scaled, mae_scaled])

In [ ]:
print(learn.summary())

## TRAIN

#### Helper functions

In [ ]:
def plot_lr_find(learn):
    learn.lr_find()
    learn.recorder.plot()

In [ ]:
def fit_one_cycle(learn, stage, n_epochs, max_lr):
    learn.fit_one_cycle(n_epochs, max_lr=max_lr, callbacks=[
        SaveModelCallback(learn,
                          monitor='mean_absolute_error',
                          mode='min',
                          every='improvement',
                          name=save_path/f'{version}-{stage}-best')])
    learn.recorder.plot_losses()
    learn.recorder.plot_lr(show_moms=True)

#### Stage 1.1

In [ ]:
learn.freeze()

In [ ]:
plot_lr_find(learn)

In [ ]:
stage = 's1.1'
n_epochs = 6
max_lr = slice(1.1e-2)

In [ ]:
fit_one_cycle(learn, stage, n_epochs, max_lr)

#### Stage 1.2

In [ ]:
learn.load(save_path/f'{version}-s1.1-best');

In [ ]:
learn.freeze()

In [ ]:
plot_lr_find(learn)

In [ ]:
stage = 's1.2'
n_epochs = 12
max_lr = slice(1e-4)

In [ ]:
fit_one_cycle(learn, stage, n_epochs, max_lr)

#### Stage 2.1

In [ ]:
learn.load(save_path/f'{version}-s1.2-best');

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find(start_lr=1e-8)
learn.recorder.plot()

In [ ]:
stage = 's2.1'
n_epochs = 6
max_lr = slice(4e-7, 4e-6)

In [ ]:
fit_one_cycle(learn, stage, n_epochs, max_lr)

#### Stage 2.2

In [ ]:
learn.load(save_path/f'{version}-s2.1-best');

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find(start_lr=1e-8)
learn.recorder.plot()

In [ ]:
stage = 's2.2'
n_epochs = 24
max_lr = slice(2e-7, 2e-6)

In [ ]:
fit_one_cycle(learn, stage, n_epochs, max_lr)

## RESULTS

In [ ]:
learn.load(save_path/f'{version}-s2.2-best');

In [ ]:
# plot histograms of results
def plot_results(dataset):
    preds, targs = learn.get_preds(ds_type=dataset)
    preds = preds * scale
    targs = targs * scale
    print('min/max pred: ', min(preds).item(), max(preds).item())
    print('min/max targ: ', min(targs).item(), max(targs).item())
    fig, ([ax1, ax2], [ax3, ax4]) = plt.subplots(nrows=2, ncols=2, figsize=(18, 12))
    ax1.hist(preds.squeeze(), bins=50); ax1.set_xlabel('TTF'); ax1.set_ylabel('preds')
    ax2.plot(preds); ax2.set_xlabel('segment')
    ax3.hist(targs, bins=50); ax3.set_xlabel('TTF'); ax3.set_ylabel('targs')
    ax4.plot(targs); ax4.set_xlabel('segment')

#### Training Set

In [ ]:
plot_results(DatasetType.Fix)

#### Validation Set

In [ ]:
plot_results(DatasetType.Valid)

#### Test Set

In [ ]:
plot_results(DatasetType.Test)

## SUBMISSION

In [ ]:
test_preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
# load sample submission
submission = pd.read_csv(path/'sample_submission.csv', index_col='seg_id')

In [ ]:
# assume order of test set is unchanged
submission['time_to_failure'] = [test_preds[i].item() * scale for i in range(len(test_preds))]

In [ ]:
submission.head()

In [ ]:
submission_file = path/f'{version}-submission.csv'
submission.to_csv(submission_file)

In [ ]:
# submit to leaderboard
! cd $path; kaggle competitions submit -c LANL-Earthquake-Prediction -f $submission_file -m "Message"

In [ ]:
# mae_scaled   : 2.157745    
# Public Score : 1.689